In [3]:
import pandas as pd
df = pd.read_csv("emnist-letters.csv")
X = df.iloc[:,1:].to_numpy().reshape(-1, 28, 28, order="F")
y = df.iloc[:,0].to_numpy()-1
print(X[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   4
    4   2   2   2   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   2   4   5   9  32
   32  21  21  20   9   4   2   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   3  21  34  82 125 129 140 203
  203 174 172 170 140 125  63  18   0   0]
 [  0   0   0   0   0   0   0   1  20  34  84 170 204 233 249 250 250 254
  254 252 252 252 250 237 111  32   0   0]
 [  0   0   0   0   0   0   0   8  77 115 172 232 245 252 254 254 254 254
  254 254 254 253 247 206  76  20   0   0]
 [  0   0   0   0   0   0   7  90 218 243 252 254 254 254 254 254 255 255
  254 254 247 207 159  47   7   1   0   0]
 [  0   0   0   0   0   3  22 159 245 253 254 254 252 251 254 25